In [1]:
import pandas as pd
output_path = 'data/combined/'
date = '240411'

In [38]:
pcs_all = pd.read_parquet(output_path+date+'_All/facilityEmissions_'+'PC4'+'_.parquet')
ints_all = pd.read_parquet(output_path+date+'_All/facilityEmissions_'+'IC3'+'_.parquet')
ds_all = pd.read_parquet(output_path+date+'_All/facilityEmissions_'+'DS2'+'_.parquet')

In [39]:
pcs_bio = pd.read_parquet(output_path+date+'_Bio/facilityEmissions_'+'PC4'+'_.parquet')
ints_bio = pd.read_parquet(output_path+date+'_Bio/facilityEmissions_'+'IC3'+'_.parquet')
ds_bio = pd.read_parquet(output_path+date+'_Bio/facilityEmissions_'+'DS2'+'_.parquet')

In [40]:
# Import chemical type list
input_path = '/data/'
product_group_path = input_path+'extra_inputs/product_groups.csv'
product_group = pd.read_csv(product_group_path, index_col=0)

In [41]:
# Import regions list
country_groupings = pd.read_csv('/data/extra_inputs/country_regions.csv')
# Correct country names
country_replace_dict = {'United States': 'USA', 'Taiwan, Province Of China':'Taiwan', 'Netherlands-The':'Netherlands', 'Russian Federation':'Russia', 'Hong Kong Special Administrative Region':'Hong Kong', 'Dominican Republ.':'Dominican Republic', 'Congo (Dem.Rep.)':'Democratic Republic of the Congo', 'Congo':'Democratic Republic of the Congo', 'Congo Republic':'Republic of the Congo', 'Slovak Republic':'Slovakia', 'Serbia/Montenegro/Kosovo':'Serbia', 'Bosnia/Herzegovina':'Bosnia and Herzegovina', 'Myanmar(Burma)':'Myanmar', 'Papua N.G.':'Papua New Guinea', 'Uk':'United Kingdom', 'Usa':'USA'}

country_groupings['COUNTRY/TERRITORY'] = country_groupings['COUNTRY/TERRITORY'].str.title().replace(country_replace_dict)

In [42]:
# Correct Benzene misnaming for Maleic Anhydride Benzene route
cols = [str(i) for i in range(1978, 2051)]
sigma_cols = [str(i)+'_sigma' for i in range(1978, 2051)]

for df in [pcs_all, pcs_bio]:
    df.loc[(df['PRODUCT'] == 'BENZENE') & (df['Type'] == 'Direct Process'), cols] = 0
    df.loc[(df['PRODUCT'] == 'BENZENE') & (df['Type'] == 'Direct Process'), sigma_cols] = 0

In [43]:
all = pd.concat([pcs_all, ints_all, ds_all])
all = all.merge(product_group, left_on='PRODUCT', right_on='PRODUCT', how='left')

all['COUNTRY/TERRITORY'] = all['COUNTRY/TERRITORY'].str.title().replace(country_replace_dict)
all = pd.merge(all, country_groupings, on='COUNTRY/TERRITORY', how='left')

all.to_parquet(output_path+date+'_All/facilityEmissions_all.parquet')

In [44]:
all_bio = pd.concat([pcs_bio, ints_bio, ds_bio])
all_bio = all_bio.merge(product_group, left_on='PRODUCT', right_on='PRODUCT', how='left')

all_bio = pd.merge(all_bio, country_groupings, on='COUNTRY/TERRITORY', how='left')
all_bio['COUNTRY/TERRITORY'] = all_bio['COUNTRY/TERRITORY'].str.title().replace(country_replace_dict)

all_bio.to_parquet(output_path+date+'_Bio/facilityEmissions_bio.parquet')

In [5]:
years = [str(i) for i in range(1978, 2051)]
years_sigma = [str(i)+'_sigma' for i in range(1978, 2051)]
keep_cols = ['PRODUCT', 'Product group', 'Product type', 'Type']

In [2]:
all = pd.read_parquet(output_path+date+'_All/facilityEmissions_all.parquet')

In [6]:
summarised = all[(all['Gas'] == 'CO2e_100a')]
summarised = summarised[keep_cols+years+years_sigma].groupby(keep_cols).sum().reset_index()

In [7]:
summarised.to_csv(output_path+date+'_All/summarised_emissions_product.csv')

In [2]:
summarised = pd.read_csv(output_path+date+'_All/summarised_emissions.csv')

In [49]:
exclude_types = ['EI & CM', 'Electricity', 'IHS CtOG']
year = 2020
gas = 'CO2e_100a'
sankey_emissions = summarised[~summarised['Type'].isin(exclude_types)]
sankey_emissions['Type'] = sankey_emissions['Type'].str.replace('Utilities', 'Energy Use')
sankey_emissions = sankey_emissions[['Product group', 'Product type', 'Type', str(year), str(year)+'_sigma']]

/var/folders/xm/c2mlyq6s23nb_g6_v4fylb7h0000gn/T/ipykernel_17401/4107309057.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sankey_emissions['Type'] = sankey_emissions['Type'].str.replace('Utilities', 'Energy Use')


In [51]:
prim_ratio = sankey_emissions[(sankey_emissions['Product group'] == 'Primary chemicals') & (sankey_emissions['Type'] != 'Primary chemicals')]['2020'].sum()/sankey_emissions[(sankey_emissions['Type'] == 'Primary chemicals') & (sankey_emissions['Product group'] != 'Primary chemicals')]['2020'].sum()

sankey_balanced = sankey_emissions.copy()
sankey_balanced['ratio'] = sankey_balanced['Type'].apply(lambda x: prim_ratio if x == 'Primary chemicals' else 1)
sankey_balanced[str(year)] = sankey_balanced[str(year)]*sankey_balanced['ratio']

ints_used = sankey_balanced[(sankey_balanced['Product type'] == 'Downstream') & (sankey_balanced['Type'] == 'Intermediates')][str(year)].sum()
ints_input = sankey_balanced[(sankey_balanced['Product group'] == 'Intermediates') & (sankey_balanced['Type'] != 'Intermediates')][str(year)].sum()
extra_ints = pd.DataFrame(['Other downstream', 'Downstream', 'Intermediates', ints_input-ints_used], index = ['Product group', 'Product type', 'Type', str(year)]).transpose()
sankey_balanced = pd.concat([sankey_balanced, extra_ints])
sankey_balanced = sankey_balanced.groupby(['Product group', 'Product type', 'Type']).sum().reset_index()
# ints_ratio = sankey_emissions[(sankey_emissions['Product group'] == 'Intermediates') & (sankey_emissions['Type'] != 'Intermediates')]['2020'].sum()/sankey_emissions[(sankey_emissions['Type'] == 'Intermediates') & (sankey_emissions['Product group'] != 'Intermediates')]['2020'].sum()
# 
# sankey_balanced['ratio'] = sankey_balanced['Type'].apply(lambda x: ints_ratio if x == 'Intermediates' else 1)
# sankey_balanced[str(year)] = sankey_balanced[str(year)]*sankey_balanced['ratio']
# sankey_balanced

In [52]:
sankey_for_markers = sankey_balanced.copy()
sankey_for_markers = sankey_for_markers[sankey_for_markers[str(year)] > 0]
basics = ['Feedstock', 'Indirect Energy Use', 'Direct Energy Use', 'Direct Process']
primary_markers = sankey_for_markers[(sankey_for_markers['Product type'] != 'Primary chemicals') & sankey_for_markers['Type'].isin(basics)].groupby(['Type']).sum().reset_index()
primary_markers['Product group'], primary_markers['Product type'] = 'First marker', 'marker'

intermediate_markers = sankey_for_markers[(sankey_for_markers['Product type'] == 'Downstream') & sankey_for_markers['Type'].isin(basics+['Primary chemicals'])].groupby(['Type']).sum().reset_index()
intermediate_markers['Product group'], intermediate_markers['Product type'] = 'Second marker', 'marker'

sankey_markers = pd.concat([sankey_balanced, primary_markers, intermediate_markers])

In [53]:
ds = ['Solvents, additives & explosives', 'Thermoplastics', 'Thermosets, fibre & elastomers', 'N-fertilisers']
all_reverse = sankey_emissions[sankey_emissions['Type'].isin(ds)]
tot_reverse = pd.DataFrame(all_reverse[all_reverse['Product group'].isin(['Primary chemicals', 'Intermediates'])].sum()).transpose()
tot_reverse['Product group'], tot_reverse['Product type'], tot_reverse['Type'] = 'Upstream', 'Upstream', 'Downstream'
int_reverse = all_reverse[all_reverse['Product type'] == 'Intermediates']
int_reverse = pd.DataFrame(int_reverse.sum()).transpose()
int_reverse['Product group'], int_reverse['Product type'], int_reverse['Type'] = 'Intermediates', 'Intermediates', 'Downstream'
prim_reverse = all_reverse[all_reverse['Product type'] == 'Primary chemicals']
prim_reverse = pd.DataFrame(prim_reverse.sum()).transpose()
prim_reverse['Product group'], prim_reverse['Product type'], prim_reverse['Type'] = 'Primary', 'Primary', 'Downstream'
sankey_with_reverse = pd.concat([sankey_markers, tot_reverse, int_reverse, prim_reverse])

In [54]:
sankey_with_capture = sankey_with_reverse.copy()
sankey_with_capture[str(year)] = sankey_with_capture[str(year)].apply(lambda x: abs(x))

scale_factor = 1E3

sankey_with_capture[str(year)] = sankey_with_capture[str(year)]/scale_factor
sankey_with_capture[str(year)+'_sigma'] = sankey_with_capture[str(year)+'_sigma']/scale_factor

In [97]:
sankey_with_capture.to_excel(output_path+date+'_All/sankey_emissions.xlsx', sheet_name='sankey_emissions')